In [ ]:
import hpelm
from sklearn.metrics import precision_recall_fscore_support

from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam

from keras.layers.core import Dense, Activation, Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Layout, Figure, Marker
import tensorflow as tf
from matplotlib import pyplot as plt

CNN_EPOCH =30
image_shape=(224,224,3)
NUM_CLASS = 2
from keras.callbacks import ReduceLROnPlateau
import time


In [ ]:
def dataload():
    train_data=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,)
    training_data= train_data.flow_from_directory("Dehazed_Images_3400/Dehazed_Images/train",target_size=(224,224),class_mode='binary' )
    training_data

    test_data=ImageDataGenerator(rescale=1/255)
    testing_data=test_data.flow_from_directory("Dehazed_Images_3400/Dehazed_Images/test",target_size=(224,224),class_mode='binary')
    return training_data,testing_data

In [ ]:
def cnn_generate(training_data,testing_data):
   
    np.random.seed(1000)
    training_data_oh = np_utils.to_categorical(training_data.classes, NUM_CLASS)
    model=Sequential()

    #1
    model.add(Conv2D(filters=96,input_shape=image_shape,kernel_size=(11,11),strides=(4,4),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))


    #2
    model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))

    #3
    model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #4
    model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #5
    model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', ))

    model.add(Flatten())
 
    #Ist fc layer 
    model.add(Dense(4096,input_shape=(224,224,3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.4))
    
    #output layer
    model.add(Dense(1))    
    model.add(BatchNormalization())
    model.add(Activation("tanh"))
    
    adam = Adam(learning_rate=0.001)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    lrr= ReduceLROnPlateau( monitor='val_accuracy',   factor=.01,   patience=3,  min_lr=1e-5) 
    
  

    t0=time.time()
    history=model.fit(training_data,epochs=CNN_EPOCH,validation_data=testing_data,validation_steps=2,callbacks = [lrr])
    print("Training time :" , time.time()-t0)
    #accuracy vs validation accuracy
    p1=plt
    p1.plot(history.history['accuracy'])
    p1.plot(history.history['val_accuracy'])
    p1.title('model accuracy')
    p1.ylabel('accuracy')
    p1.xlabel('epoch')
    p1.legend(['train', 'test'], loc='upper left')
    p1.show()


    p2=plt
    #training loss vs validation loss
    p2.plot(history.history['loss'])
    p2.plot(history.history['val_loss'])
    p2.title('model loss')
    p2.ylabel('loss')
    p2.xlabel('epoch')
    p2.legend(['train', 'test'], loc='upper left')
    p2.show()
    return model



In [ ]:
def hidden_layer_generate(cnn_model,training_data):
    layer_name = 'flatten'
    hidden_layer_model = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer(layer_name).output)

    cnn_train_result = hidden_layer_model.predict(training_data)
    #print(cnn_train_result)
    return hidden_layer_model, cnn_train_result

In [ ]:
def elm_model_generate(data_train, training_data):
    NUM_CLASS=2
    ELM_HIDDEN_NEURONS=256
    target_train_oh = np_utils.to_categorical(training_data.classes, NUM_CLASS)
    
    elm_model = hpelm.elm.ELM(data_train.shape[1], NUM_CLASS)
    elm_model.add_neurons(ELM_HIDDEN_NEURONS, func='sigm')
    #print(data_train,target_train_oh)
    elm_model.train(data_train, target_train_oh, 'c')
    
    return elm_model

In [ ]:
def plot_precision_recall_fscore(precision, recall, fscore, filename='new_result.html'):
    print("precision=",precision,"recall=",recall,"fscore=",fscore)
    avg_data = np.array([
        np.mean(precision),
        np.mean(recall),
        np.mean(fscore)
    ])

    panda_avg = pd.DataFrame(avg_data).round(3)

    data = np.array(panda_avg.values).T[0]

    print(avg_data)
    print(panda_avg)
    print(data)
    
    trace = [plotly.graph_objs.Bar(
        x=['precision', 'recall', 'fscore'],
        y=data,
        marker=Marker(color='#04b486'),
        text=data,
        textposition='auto',
        textfont=dict(color='black', size=20)
    )]

    layout = Layout(bargap=0.6, plot_bgcolor='#E6E6E6')
    plot_data = Figure(data=trace, layout=layout)
    plotly.offline.plot(plot_data, filename=filename)